In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from scipy.sparse import csr_matrix
from itertools import product, chain
import functools 
import operator 
from csv import reader
import regex as re
import time

from classifiers import *
from metrics import *
from kernels import *

from sklearn.model_selection import train_test_split # lui il va partir mais pour l'instant c'est pratique

# Load the data

In [2]:
def features_into_array(path):
    with open(path, 'r') as read_obj:
        csv_reader = reader(read_obj)
        header = next(csv_reader)
        X = list()
        if header != None:
            for row in csv_reader:
                # row variable is a list that represents a row in csv
                X.append(np.array(row[1]))
                
    X = np.array(X) ## dtype might be changed in something more convenient. For now, dtype = "<U1"
    return X

In [3]:
Xtr0 = features_into_array("data/Xtr0.csv")
Ytr0 = np.genfromtxt("data/Ytr0.csv", delimiter=',', skip_header=1)

Xtr1 = features_into_array("data/Xtr1.csv")
Ytr1 = np.genfromtxt("data/Ytr1.csv", delimiter=',', skip_header=1)

Xtr2 = features_into_array("data/Xtr2.csv")
Ytr2 = np.genfromtxt("data/Ytr2.csv", delimiter=',', skip_header=1)

In [4]:
def accuracy(y_true,y_pred, mode='SVM'):
    n = y_true.shape[0]
    if mode == 'SVM':
        predictions = np.ones(n)
        predictions[y_pred < 0] = 0
    else:
        predictions = np.zeros(n)
        predictions[y_pred >= 0.5] = 1
    
    return np.sum(y_true == predictions) / n

In [5]:
Xtr0[0]

'TCCTGTGCACATCTGCACCCCTGTTGTGGCCACAAAATGATCCGGCACCACCCAGTGGGAGACGACAGAGGTGGCAATGGGGTGTCGGCTCTGACGCCTCC'

## Mismatch Spectrum kernel

For a fixed value k (that needs to be tuned), the k-spectrum kernel is defined as : 


\begin{align*}
K(x,x^{\prime}) := \sum_{u \in \mathcal{A}^k} \phi_{u}(x) \phi_{u}(x^{\prime})
\end{align*}

We relax this constraint by authorizing each word of the alphabet to have up to m mismatches.

## Finding the neighbors

In [6]:
def neighbors(word, m):
    """
    This gives neighbors that differ in exactly m places
    """
    
    char_list = list(['A', 'C','G','T'])
    assert(m <= len(word))

    if m == 0:
        return [word]

    r2 = neighbors(word[1:], m-1)
    r = [c + r3 for r3 in r2 for c in char_list if c != word[0]]

    if (m < len(word)):
        r2 = neighbors(word[1:], m)
        r += [word[0] + r3 for r3 in r2]

    return r

def neighbors2(pattern, m):
    """
    This gives neighbors that differ in at most m places.
    """
    return sum([neighbors(pattern, d2) for d2 in range(m + 1)], [])


## Creating the possible substrings

In [7]:
def all_possible_substrings(k):
    """
    With a k spectrum kernel, let us find all the possible combinations of chars of size k in the sequence x
    This way, we could index them in the sequence x
    """
    char_list = list(['A', 'C','G','T'])
    alphabet_tuples = list(product(char_list,repeat=k))
    alphabet = dict()
    idx=0
    for i in alphabet_tuples:
        alphabet[functools.reduce(operator.add, (i))] = idx
        idx += 1
        #alphabet.append(functools.reduce(operator.add, (i)))
    return alphabet

In [8]:
def all_possible_substrings_mismatch2(k,m):
    """
    With a k spectrum kernel, let us find all the possible combinations of chars of size k in the sequence x
    This way, we could index them in the sequence x
    """
    
    alphabet = all_possible_substrings(k)
    alphabet_mismatch = dict()
    
    for key, value in alphabet.items():
        neighbors_key = neighbors2(key, m)
        # eah key in the dictionary receives as value a list of its own
        # and its neighbors indexes on the matrix D.
        alphabet_mismatch[key] = [alphabet[neigh] for neigh in neighbors_key]
    
    
    """
    char_list = list(['A', 'C','G','T'])
    alphabet_tuples = list(product(char_list,repeat=k))
    alphabet = list()
    for i in alphabet_tuples:
        word = functools.reduce(operator.add, (i))
        l= [word]+neighbors2(word,m)[1:]
        alphabet.append(l)
        """
    
    return alphabet_mismatch

In [9]:
def pre_indexing_mismatch2(X, k, m, alphabet=None):
    """
    Transforms an input array into a sparse matrix encoding the number of occurences of each letter of
    the alphabet composed of substrings of size k
    """
    n = X.shape[0]
    
    if alphabet is None:
        alphabet = all_possible_substrings_mismatch2(k,m)
    
    D = np.zeros((n,len(alphabet)))
    
    for i in range(X.shape[0]):
        idx=0
        while idx + k < len(X[i]):
            D[i, alphabet[X[i][idx:idx+k]]] += 1
            idx += 1
    D = csr_matrix(D, dtype = int)
    return D

In [10]:
def mismatch_spectrum_kernel2(X_train, X_val, X_test, k, m=1, alphabet =None):
    """
    Computes the spectrum kernels for X_train (n_train x n_train) and X_validation (on the RKHS generated(?) by
    X_train's samples) which is of shape n_validation x n_train
    "test" mode only gives as output the testing kernel
    """
    if alphabet is None:
        alphabet = all_possible_substrings_mismatch(k,m)
    
    D_train = pre_indexing_mismatch2(X_train, k, m, alphabet)
    D_val = pre_indexing_mismatch2(X_val, k, m, alphabet)
    D_test = pre_indexing_mismatch2(X_test, k, m, alphabet)
    
    K_train = D_train.dot(D_train.transpose())
    K_train = K_train.toarray().astype('float')
    
    K_val = D_val.dot(D_train.transpose())
    K_val = K_val.toarray().astype('float')
    
    K_test = D_test.dot(D_train.transpose())
    K_test = K_test.toarray().astype('float')
    
        
    return(K_train, K_val, K_test)

# Application on data

## Split the data

In [19]:
Xtr_0, Xval_0, ytr0, yval0 = train_test_split(Xtr0, Ytr0, test_size=0.2, random_state=42)
Xtr_1, Xval_1, ytr1, yval1 = train_test_split(Xtr1, Ytr1, test_size=0.2, random_state=42)
Xtr_2, Xval_2, ytr2, yval2 = train_test_split(Xtr2, Ytr2, test_size=0.2, random_state=42)

Xte0 = features_into_array("data/Xte0.csv")
Xte1 = features_into_array("data/Xte1.csv")
Xte2 = features_into_array("data/Xte2.csv")

# Testing the accuracy on sequences

In [22]:
from sklearn.model_selection import GridSearchCV

In [36]:
Kernels_tr_0 = []
Kernels_val_0 = []
Kernels_te_0 = []

Kernels_tr_1 = []
Kernels_val_1 = []
Kernels_te_1 = []

Kernels_tr_2 = []
Kernels_val_2 = []
Kernels_te_2 = []
m = 1


for k in range(5,10):
    print("*"*15 + "treating k = " + str(k) + 15*"*")
    print("."*15 + "treating m = " + str(1) + 15*".")
    start_time = time.time()
    alphabet_km = all_possible_substrings_mismatch2(k,m)
    print("--- Found alphabet in %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    K_tr0, K_val0, K_te0 = mismatch_spectrum_kernel2(Xtr0_, Xval0_, Xte0, k=k, m=m, alphabet=alphabet_km)
    Kernels_tr_0 += [K_tr0]
    Kernels_val_0 += [K_val0]
    Kernels_te_0 += [K_te0]

    K_tr1, K_val1, K_te1 = mismatch_spectrum_kernel2(Xtr1_, Xval1_, Xte1, k=k, m=m, alphabet=alphabet_km)
    Kernels_tr_1 += [K_tr1]
    Kernels_val_1 += [K_val1]
    Kernels_te_1 += [K_te1]

    K_tr2, K_val2, K_te2 = mismatch_spectrum_kernel2(Xtr2_, Xval2_, Xte2, k=k, m=m, alphabet=alphabet_km)
    Kernels_tr_2 += [K_tr2]
    Kernels_val_2 += [K_val2]
    Kernels_te_2 += [K_te2]

    print("--- Computed all the kernels in %s seconds ---" % (time.time() - start_time))
    print("")
    print("")
    print("")


***************treating k = 5***************
...............treating m = 1...............
--- Found alphabet in 0.08539795875549316 seconds ---
--- Computed all the kernels in 45.866326093673706 seconds ---



***************treating k = 6***************
...............treating m = 1...............
--- Found alphabet in 0.17705869674682617 seconds ---
--- Computed all the kernels in 46.19955921173096 seconds ---



***************treating k = 7***************
...............treating m = 1...............
--- Found alphabet in 0.7163012027740479 seconds ---
--- Computed all the kernels in 36.516748905181885 seconds ---



***************treating k = 8***************
...............treating m = 1...............
--- Found alphabet in 3.165091037750244 seconds ---
--- Computed all the kernels in 45.151968002319336 seconds ---



***************treating k = 9***************
...............treating m = 1...............
--- Found alphabet in 14.69835901260376 seconds ---
--- Computed all the k

In [38]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

values_C = [j*10**i for i in range(-5,-2) for j in range(1,10)]
parameters = {'C': values_C}
svm__ = svm.SVC(kernel='precomputed')
gs_k = GridSearchCV(svm__, param_grid=parameters, refit=True, verbose=1)

for k in range(len(Kernels_tr_0)):
    
        
    print("*"*15 + "treating k = " + str(k+5) + " and m = 1" + 15*"*")
    print("")
    print("-"*15 + " treating dataset 0 "+ 15*"-")
    
    gs_k.fit(Kernels_tr_0[k], ytr0[:,1])
    print(gs_k.best_estimator_)
    print(f"training score for k = {k+5} ", gs_k.score(Kernels_tr_0[k], ytr0[:,1]))
    print(f"validation score for k = {k+5} ", gs_k.score(Kernels_val_0[k], yval0[:,1]))
    
    print("")
    print("-"*15 + " treating dataset 1 "+ 15*"-")
    
    gs_k.fit(Kernels_tr_1[k], ytr1[:,1])
    print(gs_k.best_estimator_)
    print(f"training score for k = {k+5} ", gs_k.score(Kernels_tr_1[k], ytr1[:,1]))
    print(f"validation score for k = {k+5} ", gs_k.score(Kernels_val_1[k], yval1[:,1]))
    
    print("")
    print("-"*15 + " treating dataset 2 "+ 15*"-")
    
    gs_k.fit(Kernels_tr_2[k], ytr2[:,1])
    print(gs_k.best_estimator_)
    print(f"training score for k = {k+5} ", gs_k.score(Kernels_tr_2[k], ytr2[:,1]))
    print(f"validation score for k = {k+5} ", gs_k.score(Kernels_val_2[k], yval2[:,1]))
    
    print("")
    print("")

***************treating k = 5 and m = 1***************

--------------- treating dataset 0 ---------------
Fitting 5 folds for each of 27 candidates, totalling 135 fits
SVC(C=0.0004, kernel='precomputed')
training score for k = 5  0.71875
validation score for k = 5  0.62

--------------- treating dataset 1 ---------------
Fitting 5 folds for each of 27 candidates, totalling 135 fits
SVC(C=0.00030000000000000003, kernel='precomputed')
training score for k = 5  0.73
validation score for k = 5  0.6075

--------------- treating dataset 2 ---------------
Fitting 5 folds for each of 27 candidates, totalling 135 fits
SVC(C=0.0002, kernel='precomputed')
training score for k = 5  0.779375
validation score for k = 5  0.7275


***************treating k = 6 and m = 1***************

--------------- treating dataset 0 ---------------
Fitting 5 folds for each of 27 candidates, totalling 135 fits
SVC(C=0.0002, kernel='precomputed')
training score for k = 6  0.7625
validation score for k = 6  0.67

--

# Predictions

In [ ]:
test_kernels = [K_te0, K_te1, K_te2]
#test_alphas = [alphas_tr0[-4], alphas_tr1[-4], alphas_tr2[-3]] # il faut choisir l'alpha associé à un bon lambda!
test_alphas = [alphas_tr0[9], alphas_tr1[4], alphas_tr2[8]]
write_predictions_csv(test_kernels, test_alphas, path ="data/mismatch_Ytest_sequences.csv", mode="SVM")

In [31]:
len(Kernels_tr_2)

10